In [ ]:
from sedona.spark import *
import pyspark.sql.functions as F

# 1. DEFINE CONFIGURATION
config = SedonaContext.builder() \
    .config("spark.jars.packages",
           "org.apache.sedona:sedona-spark-shaded-3.5_2.12:1.6.1,"
           "org.datasyslab:geotools-wrapper:1.6.1-28.2,"
           "org.postgresql:postgresql:42.7.3") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "2g") \
    .getOrCreate()

# 2. CREATE CONTEXT
sedona = SedonaContext.create(config)

print("✅ Spark Session Started Successfully!")
print(f"Sedona Version: {sedona.version}")

In [ ]:
# 1. CREATE DUMMY DATA (KL Coordinates)
data = [
    ("KLCC", "POINT(101.7119 3.1579)"),
    ("Menara KL", "POINT(101.7040 3.1528)")
]

# 2. LOAD INTO DATAFRAME
df = sedona.createDataFrame(data, ["name", "wkt"])

# 3. CONVERT TEXT TO GEOMETRY & BUFFER (0.01 degrees approx 1km)
df_geom = df.selectExpr("name", "ST_GeomFromWKT(wkt) as geometry")
df_buffer = df_geom.selectExpr("name", "ST_Buffer(geometry, 0.01) as buffer_zone")

print("🧪 Spatial Calculation Result:")
df_buffer.show(truncate=False)

In [ ]:
# DATABASE CREDENTIALS (Internal Docker Network)
db_url = "jdbc:postgresql://sedona-db:5432/geodb"
db_props = {
    "user": "admin",
    "password": "password123",
    "driver": "org.postgresql.Driver"
}

try:
    # 1. PREPARE DATA FOR SAVING (Convert Geometry back to Text for DB)
    # PostGIS needs geometries, but Spark JDBC is safer sending Strings first
    df_save = df_buffer.selectExpr("name", "ST_AsText(buffer_zone) as geom_wkt")

    # 2. WRITE TO DB
    print("⏳ Writing to Database...")
    df_save.write.mode("overwrite").jdbc(db_url, "kl_buffers", properties=db_props)
    print("✅ Write Success! Table 'kl_buffers' created.")

    # 3. READ FROM DB
    print("⏳ Reading back from Database...")
    df_read = sedona.read.jdbc(db_url, "kl_buffers", properties=db_props)
    df_read.show(5)
    print("✅ Read Success! Connection is solid.")

except Exception as e:
    print(f"❌ Database Error: {e}")